In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import chain
from langchain_core.prompts import load_prompt

from dotenv import load_dotenv
load_dotenv()

True

### 데이터셋

In [2]:
import os
import json
import pandas as pd

with open('../data/document/역도/documentParseGraph_state.json', 'r') as f:
    state = json.load(f)

In [3]:
texts = []
headings = []

for i, document in enumerate(state['documents']):
    headings.append('\n'.join([value for key, value in document['meta']['heading'].items() if value != None]))
    texts.append(' '.join([content['text'] for content in document['content'] if content['category'] == 'paragraph']))

df = pd.DataFrame({'heading': headings, 'text': texts})
df['isNull'] = df['text'].apply(lambda x : True if x == '' else False)
df['originContext'] = df['heading'] + ' ' + df['text']

In [4]:
df.head(2)

,heading,text,isNull,originContext
0,Ⅴ.역도 훈련프로그램 구성 및 지도안\n1. 훈련프로그램의 구성 원리,역도 훈련프로그램의 구성은 트레이닝의 원리에 입각하여 안전하고 효과적인 훈\n련 프...,False,Ⅴ.역도 훈련프로그램 구성 및 지도안\n1. 훈련프로그램의 구성 원리 역도 훈련프로...
1,Ⅴ.역도 훈련프로그램 구성 및 지도안\n1. 훈련프로그램의 구성 원리\n가. 일반적...,특수성의 원리는 특수한 변화나 결과를 초래하기 위하여 특수한 방법으로 훈련\n하는 ...,False,Ⅴ.역도 훈련프로그램 구성 및 지도안\n1. 훈련프로그램의 구성 원리\n가. 일반적...


### 불필요한 공백 제거 및 전처리

In [15]:
def preprocessing_result(dataset):    
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
    preprocessing_prompt = load_prompt('../prompt/preprocessing/preprocessing_20250317_01.yaml')
    preprocessing_chain = preprocessing_prompt | llm

    result = preprocessing_chain.batch(dataset, config={"max_concurrency": 5})    
    return result

In [56]:
dataset = df.loc[df['isNull'] == False]['text'].tolist()
dataset = [{'sentences': data} for data in dataset]
# result = preprocessing_result(dataset)
result_content = [value.content for value in result]

df.loc[df['isNull'] == False, 'preprocessedText'] = result_content
df.loc[df['isNull'] == True, 'preprocessedText'] = ''
df['preprocessedContext'] = df['heading'] + '\n' + df['preprocessedText']

### 전처리 유무에 따른 요약 성능 비교 

In [58]:
def summary_result(dataset):    
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
    summary_prompt = load_prompt('../prompt/summary/stuff_20250317_01.yaml')
    summary_chain = summary_prompt | llm

    result = summary_chain.batch(dataset, config={"max_concurrency": 5})    
    return result

In [ ]:
dataset = [{'text': data} for data in df['preprocessedContext']]
# result = summary_result(dataset)
result_content = [value.content for value in result]

df['preprocessedSummary'] = result_content

In [70]:
dataset = [{'text': data} for data in df['originContext']]
# result = summary_result(dataset)
result_content = [value.content for value in result]

df['originSummary'] = result_content

### 결과 확인
1. null value(paragraph가 없는 chunking)
   - paragraph가 없고 heading만 있는 chunking의 요약문 결과를 보면, heading을 기반으로 추가적인 정보가 요약에 들어감을 확인
   - 요약 프롬프트 개선을 통해 추가정보 차단 필요
2. 전처리 유무에 따른 요약 결과 비교
   - 전처리 유무에 따른 요약 결과의 차이는 거의 없음

In [79]:
for i, row in df.loc[df['isNull'] == True].iterrows():
    print(row['preprocessedContext'])
    print(row['preprocessedSummary'])
    print(row['originSummary'])
    print('--------------------------------')


Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
라. 경력에 따른 훈련프로그램
2) 중급훈련계획(중급자)
나) 2단계

역도 훈련프로그램은 훈련의 경력에 따라 구성되어 있으며, 중급자를 위한 훈련계획은 2단계로 나뉘어 있다. 이러한 구성 원리는 훈련의 효과를 극대화하기 위해 설계되었으며, 각 단계는 훈련자의 능력에 맞춰 조정된다. 중급자에게 적합한 훈련은 기술 향상과 체력 강화를 동시에 목표로 하여, 체계적인 접근이 필요하다.
역도 훈련프로그램은 훈련의 경력에 따라 구성되어 있으며, 중급자를 위한 훈련계획이 포함되어 있다. 특히, 중급자에게 적합한 2단계 훈련이 제시되어 있어, 이 단계에서는 보다 체계적이고 효과적인 훈련을 통해 기술과 체력을 향상시키는 데 중점을 두고 있다. 이러한 구성 원리는 훈련의 효과성을 높이기 위한 중요한 요소로 작용한다.
--------------------------------
Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
마. 훈련지도안 작성
1) 대표적인 훈련계획(typical schemes for sessions)
가) 인상 기술 향상을 위한 프로그램

역도 훈련 프로그램은 효과적인 훈련을 위해 구성 원리를 따르며, 이를 바탕으로 훈련 지도안을 작성하는 것이 중요하다. 특히, 인상 기술 향상을 위한 프로그램이 대표적인 훈련 계획으로 제시된다. 이러한 프로그램은 선수의 기술적 능력을 높이는 데 중점을 두고 설계되어, 훈련의 효율성을 극대화하는 데 기여한다.
역도 훈련 프로그램은 효과적인 훈련을 위해 구성 원리를 따르며, 이를 바탕으로 훈련 지도안을 작성하는 것이 중요하다. 특히, 대표적인 훈련 계획 중 하나는 인상 기술 향상을 위한 프로그램이다. 이 프로그램은 선수들이 기술을 개선하고 성과를 높일 수 있도록 체계적으로 설계되어 있다.
--------------------------------
Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
마. 훈련지도안 작성
1) 대

### summary 수정
- paragraph가 없는 chunking 요약 결과에 추가 정보 조치 방안 추가

In [82]:
def summary_result(dataset):    
    llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)
    summary_prompt = load_prompt('../prompt/summary/stuff_20250317_02.yaml')
    summary_chain = summary_prompt | llm

    result = summary_chain.batch(dataset, config={"max_concurrency": 5})    
    return result

In [84]:
dataset = [{'text': data} for data in df['preprocessedContext']]
result = summary_result(dataset)
result_content = [value.content for value in result]

df['preprocessedSummary2'] = result_content

In [85]:
for i, row in df.loc[df['isNull'] == True].iterrows():
    print(row['preprocessedContext'])
    print(row['preprocessedSummary'])
    print(row['preprocessedSummary2'])
    print('--------------------------------')


Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
라. 경력에 따른 훈련프로그램
2) 중급훈련계획(중급자)
나) 2단계

역도 훈련프로그램은 훈련의 경력에 따라 구성되어 있으며, 중급자를 위한 훈련계획은 2단계로 나뉘어 있다. 이러한 구성 원리는 훈련의 효과를 극대화하기 위해 설계되었으며, 각 단계는 훈련자의 능력에 맞춰 조정된다. 중급자에게 적합한 훈련은 기술 향상과 체력 강화를 동시에 목표로 하여, 체계적인 접근이 필요하다.
Summary: 이 텍스트는 역도 훈련프로그램 구성 및 지도안에 대한 내용을 다루고 있다. 첫 번째로 훈련프로그램의 구성 원리에 대해 설명하며, 경력에 따른 훈련프로그램을 포함하고 있다. 두 번째로 중급훈련계획에 대한 내용이 있으며, 중급자를 위한 2단계가 언급된다.
--------------------------------
Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
마. 훈련지도안 작성
1) 대표적인 훈련계획(typical schemes for sessions)
가) 인상 기술 향상을 위한 프로그램

역도 훈련 프로그램은 효과적인 훈련을 위해 구성 원리를 따르며, 이를 바탕으로 훈련 지도안을 작성하는 것이 중요하다. 특히, 인상 기술 향상을 위한 프로그램이 대표적인 훈련 계획으로 제시된다. 이러한 프로그램은 선수의 기술적 능력을 높이는 데 중점을 두고 설계되어, 훈련의 효율성을 극대화하는 데 기여한다.
Summary: 이 텍스트는 역도 훈련프로그램 구성 및 지도안에 대한 내용을 다루고 있다. 훈련프로그램의 구성 원리에 대해 설명하며, 훈련지도안 작성과 관련된 내용을 포함하고 있다. 또한, 대표적인 훈련계획으로 인상 기술 향상을 위한 프로그램이 언급된다.
--------------------------------
Ⅴ.역도 훈련프로그램 구성 및 지도안
1. 훈련프로그램의 구성 원리
마. 훈련지도안 작성
1) 대표적인 훈련계획(typical schemes for sessions)
나) 근력발달과 